In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("Metazygia monitor 1.csv")
df = df[6904:]
display(df.head())

,Date_Time,lights,Monitor 1 Spider 1,Monitor 1 Spider 2,Monitor 1 Spider 3,Monitor 1 Spider 4,Monitor 1 Spider 5,Monitor 1 Spider 6,Monitor 1 Spider 7,Monitor 1 Spider 8,...,Monitor 1 Spider 23,Monitor 1 Spider 24,Monitor 1 Spider 25,Monitor 1 Spider 26,Monitor 1 Spider 27,Monitor 1 Spider 28,Monitor 1 Spider 29,Monitor 1 Spider 30,Monitor 1 Spider 31,Monitor 1 Spider 32
6904,2017-04-25 19:04:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6905,2017-04-25 19:05:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6906,2017-04-25 19:06:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6907,2017-04-25 19:07:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6908,2017-04-25 19:08:00,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
from scipy.stats import ttest_ind

df = df.apply(pd.to_numeric, errors='coerce')

from datetime import timedelta
# Set the threshold for activity
activity_threshold = 0.000001
min_active_duration = 18 * 60  # 18 hours in minutes
max_active_duration = 36 * 60  # 36 hours in minutes
# Find the start and end indices of each active period
active_periods = []
for spider in df.columns:
    active = df[spider] > activity_threshold
    starts = active.index[active & ~active.shift(fill_value=False)].tolist()
    ends = active.index[~active & active.shift(fill_value=False)].tolist()
    # Check if there is no ending period
    if len(starts) > len(ends):
        ends.append(df.index[-1])
    # Filter active periods within the desired duration range
    filtered_periods = []
    for start, end in zip(starts, ends):
        duration = (end - start)
        if min_active_duration <= duration <= max_active_duration:
            filtered_periods.append((start, end))
    # Store the filtered active periods
    active_periods.append(filtered_periods)
# Calculate the duration of the free-running period for each spider
for i, spider in enumerate(df.columns):
    print(f"Spider: {spider}")
    for start, end in active_periods[i]:
        duration = (end - start)
        print(f"Active period: {start} to {end} (Duration: {duration} minutes)")

Spider: Date_Time
Spider: lights
Spider: Monitor 1 Spider 1
Spider: Monitor 1 Spider 2
Spider: Monitor 1 Spider 3
Spider: Monitor 1 Spider 4
Spider: Monitor 1 Spider 5
Spider: Monitor 1 Spider 6
Spider: Monitor 1 Spider 7
Spider: Monitor 1 Spider 8
Spider: Monitor 1 Spider 9
Spider: Monitor 1 Spider 10
Spider: Monitor 1 Spider 11
Spider: Monitor 1 Spider 12
Spider: Monitor 1 Spider 13
Spider: Monitor 1 Spider 14
Spider: Monitor 1 Spider 15
Spider: Monitor 1 Spider 16
Spider: Monitor 1 Spider 17
Spider: Monitor 1 Spider 18
Spider: Monitor 1 Spider 19
Spider: Monitor 1 Spider 20
Spider: Monitor 1 Spider 21
Spider: Monitor 1 Spider 22
Spider: Monitor 1 Spider 23
Spider: Monitor 1 Spider 24
Spider: Monitor 1 Spider 25
Spider: Monitor 1 Spider 26
Spider: Monitor 1 Spider 27
Spider: Monitor 1 Spider 28
Spider: Monitor 1 Spider 29
Spider: Monitor 1 Spider 30
Spider: Monitor 1 Spider 31
Spider: Monitor 1 Spider 32
